# Adaptive RAG

In [ ]:
### Build Index
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import LanceDB
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


###### router import
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from typing import List
from langgraph.graph import END, StateGraph

import os
from dotenv import load_dotenv
load_dotenv(override=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Build DB

In [2]:
embedding_function = OpenAIEmbeddings()

urls = [
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
    ]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

 # Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    embedding=embedding_function
)
retriever = vectorstore.as_retriever()

print("vectoselected",vectorstore)
retriever = vectorstore.as_retriever()

vectoselected <langchain_community.vectorstores.chroma.Chroma object at 0x12f96ffe0>


### Router

In [ ]:
### Build Index
### Router

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""
    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """你是一個專家，負責將用戶問題路由到向量資料庫或網絡搜索。
向量資料庫包含與代理、提示工程和對抗性攻擊相關的文檔。
對於這些主題的問題使用向量資料庫。否則，使用網絡搜索。"""
route_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{question}")])
question_router = route_prompt | structured_llm_router


question1 ="李多慧什麼時候加入統一獅？"
print(question_router.invoke({"question": question1}))
print(question_router.invoke({"question": "請問 Agent 是什麼？"}))


datasource='web_search'
datasource='vectorstore'


### Retrieval Grader
這個組件用於評估檢索文檔與問題的相關性：

In [4]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """你是一個評分員，評估檢索文檔與用戶問題的相關性。
如果文檔包含與用戶問題相關的關鍵詞或語義含義，將其評為相關。
這不需要是嚴格的測試。目的是過濾掉錯誤的檢索結果。
給出二元評分 'yes' 或 'no' 來表示文檔是否與問題相關。"""
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "檢索文檔：\n\n {document} \n\n 用戶問題：{question}")])

retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)

doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

/var/folders/yw/v6fz89fx5pd827bs1070ywr00000gn/T/ipykernel_73111/999480785.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


binary_score='no'


In [ ]:
class Document:
    def __init__(self, page_content):
        self.page_content = page_content
    def __repr__(self):
        return f"Document(page_content='{self.page_content}')"

cleaned_docs_format = [Document(doc.page_content) for doc in docs]

### Generate Node
用途：生成節點負責基於檢索的文檔生成答案

In [ ]:
# Prompt
prompt = ChatPromptTemplate.from_messages([
("human", """你是一個問答任務的助手。使用以下檢索的上下文來回答問題。如果你不知道答案，就說不知道。最多使用三個句子，保持答案簡潔。
Question: {question}
Context: {context}
Answer:""")])

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# # Run
generation = rag_chain.invoke({"context": cleaned_docs_format, "question": question})
print(generation)

The agent memory includes a long-term memory module that records agents' experiences in natural language. It combines memory, planning, and reflection mechanisms to enable agents to behave based on past experiences and interact with other agents. The retrieval model surfaces context based on relevance, recency, and importance to inform the agent's behavior.


### Hallucination Grader
這個組件用於評估生成的答案是否基於事實：

In [7]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""
    binary_score: str = Field(description="Answer is grounded in the facts, 'yes' or 'no'")

# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """你是一個評分員，評估 LLM 生成的內容是否基於/支持於一組檢索的事實。
給出二元評分 'yes' 或 'no'。'Yes' 表示答案基於/支持於這組事實。"""
hallucination_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "事實集：\n\n {documents} \n\n LLM 生成：{generation}")])

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": cleaned_docs_format, "generation": generation})

GradeHallucinations(binary_score='yes')

### Answer Grader
用於評估生成的答案是否解決了問題：

In [8]:
# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""
    binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")

# LLM with function call
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """你是一個評分員，評估答案是否解決/回答了問題。
給出二元評分 'yes' 或 'no'。"""
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "用戶問題：\n\n {question} \n\n LLM 生成：{generation}")])

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question,"generation": generation})

GradeAnswer(binary_score='no')

### Question Re-writer
這個組件用於改寫用戶問題，使其更適合檢索：

In [ ]:
# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Prompt
system = """你是一個問題重寫器，將輸入問題轉換為更適合向量資料庫檢索的版本。
分析輸入並嘗試理解其底層語義意圖/含義。"""
re_write_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "這是初始問題：\n\n {question} \n 制定一個改進的問題。")])

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'改寫後的問題：\n\n如何有效地管理代理人的記憶？'

In [10]:
### Search
web_search_tool = TavilySearchResults(k=3)

### Define State

In [11]:
#################
#langgraph code

class GraphState(TypedDict):
    question : str
    generation : str
    documents : List[str]


## Define Nodes

包括檢索、生成、文檔評分、查詢轉換和網絡搜索。這些函數將在我們的圖中作為節點使用。

In [12]:
## graph flow
def retrieve(state):
    print("--- RETRIEVE DOCS ---")
    question = state["question"]
    documents = retriever.invoke(question)

    # docs = retriever.get_relevant_documents(question)
    # documents = [doc.page_content for doc in docs]

    # docsNEW = [doc.page_content for doc in docs]  # Adjust if the actual attribute name differs
    print("TAKING DICSNEW >>>>>>>>")
    return {"documents": cleaned_docs_format, "question": question}

def generate(state):
    print("--- GENERATE ---")
    question = state["question"]
    documents = state["documents"]

    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    print("--- GRADE DOCS ---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("--- [GRADE]: RELEVANCE ---")
            filtered_docs.append(d)
        else:
            print("--- [GRADE]: NOT RELEVANCE ---")
            continue
    return {"documents": filtered_docs, "question": question}

def transform_query(state):
    print("--- [RE-WRITE] ---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

def web_search(state):
    print("--- WEB SEARCH ---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Define Conditional Edge
用於決定下一步應該執行哪個節點。

In [13]:
### Edges ###

def route_question(state):
    print("--- ROUTING ---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == 'web_search':
        print("--- ROUTING -> WEB SEARCH ---")
        return "web_search"
    elif source.datasource == 'vectorstore':
        print("--- ROUTING -> RAG ---")
        return "vectorstore"

def decide_to_generate(state):
    print("--- ASSESS GRADED DOCUMENTS ---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("--- DECISION: RE-WIRTE ---")
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("--- DECISION: GENERATE ---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    print("--- CHECK HALLUCINATIONS ---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("--- DECISION: GENERATION IS GROUNDED IN DOCUMENTS ---")
        print("--- GRADE GENERATION vs QUESTION ---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("--- DECISION: GENERATION ADDRESSES QUESTION ---")
            return "useful"
        else:
            print("--- DECISION: GENERATION DOES NOT ADDRESS QUESTION ---")
            return "not useful"
    else:
        print("--- DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY ---")
        return "not supported"


### Build Graph


In [14]:
## grapj build
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae
workflow.add_node("transform_query", transform_query) # transform_query

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

### Inference

In [19]:
# Run
inputs = {"question": "什麼是 Agent ?"}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Node '{key}':")
    print('')
# Final generation
print(value["generation"])

--- ROUTING ---
--- ROUTING -> RAG ---
--- RETRIEVE DOCS ---
TAKING DICSNEW >>>>>>>>
Node 'retrieve':

--- GRADE DOCS ---
--- [GRADE]: RELEVANCE ---
--- [GRADE]: RELEVANCE ---
--- [GRADE]: RELEVANCE ---
--- [GRADE]: RELEVANCE ---
--- ASSESS GRADED DOCUMENTS ---
--- DECISION: GENERATE ---
Node 'grade_documents':

--- GENERATE ---
--- CHECK HALLUCINATIONS ---
--- DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY ---
Node 'generate':

--- GENERATE ---
--- CHECK HALLUCINATIONS ---
--- DECISION: GENERATION IS GROUNDED IN DOCUMENTS ---
--- GRADE GENERATION vs QUESTION ---
--- DECISION: GENERATION DOES NOT ADDRESS QUESTION ---
Node 'generate':

--- [RE-WRITE] ---
Node 'transform_query':

--- RETRIEVE DOCS ---
TAKING DICSNEW >>>>>>>>
Node 'retrieve':

--- GRADE DOCS ---
--- [GRADE]: RELEVANCE ---
--- [GRADE]: RELEVANCE ---
--- [GRADE]: RELEVANCE ---
--- [GRADE]: RELEVANCE ---
--- ASSESS GRADED DOCUMENTS ---
--- DECISION: GENERATE ---
Node 'grade_documents':

--- GENERATE ---
--- CHECK 

In [17]:
# Run
inputs = {"question": "誰是美國建國總統？"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node//
        print(f"Node '{key}':")
    print('')

# Final generation
print(value["generation"])

--- ROUTING ---
--- ROUTING -> WEB SEARCH ---
--- WEB SEARCH ---
Node 'web_search':

--- GENERATE ---
--- CHECK HALLUCINATIONS ---
--- DECISION: GENERATION IS GROUNDED IN DOCUMENTS ---
--- GRADE GENERATION vs QUESTION ---
--- DECISION: GENERATION ADDRESSES QUESTION ---
Node 'generate':

美國建國總統是喬治·華盛頓。
